In [2]:
from kafka import KafkaConsumer
import psycopg2
import os
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
PG_HOST=os.getenv("PG_HOST")
PG_PORT=os.getenv("PG_PORT")
PG_USER=os.getenv("PG_USER")
PG_PASSWORD=os.getenv("PG_PASSWORD")
PG_DATABASE=os.getenv("PG_DATABASE")

In [6]:
HOST_IP=os.getenv("HOST_IP")


In [7]:
TOPIC_NAME = os.getenv("KAFKA_TOPIC1")
KAFKA_BOOTSTRAP_SERVER = [f'{HOST_IP}:9092']

In [8]:
consumer = KafkaConsumer(
    TOPIC_NAME,
    bootstrap_servers=KAFKA_BOOTSTRAP_SERVER,
    auto_offset_reset='earliest'
)

In [9]:
conn = psycopg2.connect(
    host=PG_HOST,
    port=PG_PORT,
    user=PG_USER,
    password=PG_PASSWORD,
    database=PG_DATABASE
)
cur = conn.cursor()

In [10]:
insert_query = """
INSERT INTO metrics (id, cpu, mem, disk)
VALUES (%s, %s, %s, %s)
"""

In [11]:
for message in consumer:
    data = message.value

    value = message.value.decode('utf-8')
    numbers = tuple([int(item.split(":")[1].strip()) for item in value.split(",")])
 
    # Insert data into PostgreSQL table
    cur.execute(insert_query,numbers)
    conn.commit()

    print(f"Received message:value = {numbers}")


Received message:value = (2, 9, 100, 52)
Received message:value = (8, 28, 93, 26)
Received message:value = (0, 15, 76, 42)
Received message:value = (3, 98, 34, 49)
Received message:value = (6, 55, 94, 86)
Received message:value = (4, 17, 90, 37)
Received message:value = (7, 26, 27, 61)
Received message:value = (1, 71, 38, 37)
Received message:value = (5, 73, 21, 2)
Received message:value = (9, 93, 76, 61)
Received message:value = (8, 41, 93, 63)
Received message:value = (7, 91, 28, 25)
Received message:value = (2, 83, 43, 45)
Received message:value = (5, 20, 46, 27)
Received message:value = (3, 78, 81, 8)
Received message:value = (0, 57, 50, 35)
Received message:value = (6, 96, 78, 6)
Received message:value = (1, 82, 44, 53)
Received message:value = (9, 59, 85, 21)
Received message:value = (4, 19, 60, 48)
Received message:value = (8, 54, 39, 9)
Received message:value = (7, 100, 75, 21)
Received message:value = (5, 95, 42, 29)
Received message:value = (6, 23, 51, 46)
Received message:va

KeyboardInterrupt: 

In [12]:
# Close connections
consumer.close()
cur.close()
conn.close()